### **Notebook Objectives**

1. **Train Neural Codecs:** Train both the Baseline autoencoder and the carbon-aware EQODEC model on the Vimeo-90K training split using deterministic hyperparameters.

2. **Validate Per Epoch:** Monitor reconstruction quality and rate–distortion behavior by computing validation loss and PSNR during each epoch.

3. **Evaluate Sustainability:** Compute the Energy-Efficiency Score (EES) every epoch using FFmpeg compression and CodeCarbon to quantify carbon overhead, bitrate savings, and energy–quality trade-offs.

4. **Select Best Models:** Automatically save the best-performing checkpoint for each replicate based on validation loss.

5. **Run Robust Experiments:** Perform multiple training replicates (N = 5) for statistical reliability, then compute mean and standard deviation for all key metrics.

6. **Generate Final Comparison:** Produce aggregated Baseline vs. EQODEC results—including EES, PSNR, overhead time, compressed size, and CO₂ emissions—and export them for downstream analysis.

### **Model Training & Evaluation**

In [1]:
# Standard Library Imports
import os
import math
import json
import shutil
import tempfile
import time
import subprocess
import traceback
from typing import Optional, Tuple, Dict, List

# Third-Party Library Imports
import numpy as np
import pandas as pd
from PIL import Image
from tqdm import tqdm
from codecarbon import EmissionsTracker

# PyTorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from torch.amp import autocast, GradScaler

# Other Utilities
import warnings
warnings.filterwarnings("ignore")

In [2]:
# Ensure directories exist
model_dir = "../models"
results_dir = "../results"
os.makedirs(model_dir, exist_ok=True)
os.makedirs(results_dir, exist_ok=True)

## --- Config Constants ---
FFMPEG = "ffmpeg"
FFMPEG_CRF = 23 # Constant Rate Factor for the Baseline (Original)
FFMPEG_MODEL_CRF = 32 # CRF for the Model's Compressed Proxy (Higher CRF = Lower Quality/Size)
FFMPEG_PRESET = "medium"

# Vimeo Native Resolution
EVAL_RES_W = 448
EVAL_RES_H = 256
EVAL_CHUNK_SIZE = 4

# ARCHITECTURE CONSTANTS
MODEL_BASE_CHANNELS = 16

# TRAINING CONSTANTS
N_RUNS = 5 # Number of replicates for robust evaluation
LAMBDA_RECON = 5.0 # Reconstruction loss term weight (MSE)
LAMBDA_RATE = 0.5 # Rate-Distortion (R-D) BPP term weight
LAMBDA_C_EQODEC = 0.005 # Carbon loss term weight (EQODEC)
LAMBDA_C_BASELINE = 0.0 # Carbon loss term weight (Baseline)

## --- Utility Serializer ---
def default_serializer(obj):
    """Custom JSON serializer for NumPy and torch floats."""
    if isinstance(obj, (np.float32, np.float64, float)):
        return float(obj)
    if isinstance(obj, (torch.Tensor, np.ndarray)):
        return obj.tolist()
    return str(obj)

## --- EQODEC Carbon Module ---
class CarbonModule:
    """Independent carbon utilities using CodeCarbon logic."""

    @staticmethod
    def carbon_proxy(latent: torch.Tensor, carbon_intensity: float):
        """
        Latent -> Carbon proxy (A BPP-like term scaled by carbon intensity).
        The term log1p(abs(latent)) serves as a stable proxy for information content.
        
        Carbon Intensity is expected in kgCO2/kWh. We scale the BPP proxy by this intensity
        for an environmentally-aware regularization term.
        """
        # The loss term is proportional to BPP and carbon intensity
        # BPP proxy: torch.mean(torch.log1p(torch.abs(latent))) / math.log(2)
        # We use a similar proxy, scaled by intensity, as a regularization term.
        return torch.mean(torch.log1p(torch.abs(latent))) * carbon_intensity

    @staticmethod
    def tracker(project_name="eqodec_step", measure_power_secs=1):
        """Wrapper for CodeCarbon tracker."""
        t = EmissionsTracker(
            project_name=project_name,
            log_level="error",
            measure_power_secs=measure_power_secs,
            save_to_file=False
        )
        t.start()
        return t

## --- Dataset Class (Vimeo) ---
class VimeoDataset(Dataset):
    """Vimeo-90K septuplet loader."""
    def __init__(self, root, index_file, transform=None):
        self.root = root
        self.transform = transform
        with open(index_file, "r") as f:
            self.seqs = json.load(f)

    def __len__(self):
        return len(self.seqs)

    def __getitem__(self, idx):
        seq = self.seqs[idx]
        path = os.path.join(self.root, seq)
        frames = []
        target_size = (EVAL_RES_W, EVAL_RES_H)
        
        for i in range(1, 8):
            img = Image.open(os.path.join(path, f"im{i}.png")).convert("RGB")
            # Ensure input size matches the target evaluation size
            if img.size != target_size:
                img = img.resize(target_size, Image.Resampling.BILINEAR)

            frames.append(self.transform(img) if self.transform else transforms.ToTensor()(img))
            
        return torch.stack(frames), seq

## --- Model: ConvGRU Autoencoder ---
class ConvGRUCell(nn.Module):
    """Convolutional GRU Cell implementation (Cheaper than ConvLSTM)."""
    def __init__(self, in_ch, hid_ch, k=3):
        super().__init__()
        pad = k // 2
        self.hid_ch = hid_ch
        # r, z gates
        self.conv_rz = nn.Conv2d(in_ch + hid_ch, 2 * hid_ch, k, padding=pad)
        # n gate
        self.conv_n = nn.Conv2d(in_ch + hid_ch, hid_ch, k, padding=pad)

    def forward(self, x, h):
        # r (reset) and z (update) gates
        rz = self.conv_rz(torch.cat([x, h], 1))
        r, z = torch.split(rz, self.hid_ch, 1)
        r, z = torch.sigmoid(r), torch.sigmoid(z)
        
        # n (candidate hidden state) gate
        h_tilde = self.conv_n(torch.cat([x, r * h], 1))
        n = torch.tanh(h_tilde)
        
        # Next Hidden State
        h_next = (1 - z) * h + z * n
        return h_next, h_next # Returns h_next twice for ConvLSTM-like API compatibility

class ConvGRU(nn.Module):
    """Convolutional GRU module for sequence processing."""
    def __init__(self, in_ch, hid_ch):
        super().__init__()
        self.cell = ConvGRUCell(in_ch, hid_ch)

    def forward(self, xseq):
        B, T, C, H, W = xseq.shape
        # Initialize hidden state (h) - c is not used
        h = torch.zeros(B, self.cell.hid_ch, H, W, device=xseq.device)
        outs = []
        
        # Process sequence time step by time step
        for t in range(T):
            # Pass h and c placeholder for ConvLSTM compatibility, but c is ignored
            h, _ = self.cell(xseq[:, t], h)
            outs.append(h)
            
        return torch.stack(outs, 1)

class LatentQuantizer(nn.Module):
    """
    A simple straight-through quantizer. 
    It rounds the latent representation and scales it back for use in the decoder.
    This acts as a proxy for the actual compression bitrate.
    """
    def __init__(self, scale=64.0):
        super().__init__()
        self.scale = scale

    def forward(self, x):
        # Handle 5D (B,T,C,H,W) input from ConvGRU for quantization
        was_5d = (x.ndim == 5)
        if was_5d:
            B,T,C,H,W = x.shape
            x = x.view(B*T, C, H, W)
            
        # Quantization: round(x * scale) / scale
        # The key is to ensure gradient is passed through using the straight-through estimator (STE)
        xq = (torch.round(x * self.scale) / self.scale) + (x - x).detach()
        
        if was_5d:
            xq = xq.view(B,T,C,H,W)
            
        return xq

class EQODECAutoencoder(nn.Module):
    """
    The main EQODEC architecture: 
    Encoder (Conv) -> Temporal (ConvGRU) -> Quantizer -> Decoder (ConvTranspose)
    """
    def __init__(self, base=MODEL_BASE_CHANNELS): # Use the reduced channel count
        super().__init__()
        # Spatial Encoder: Reduce H/W by 4x
        self.enc = nn.Sequential(
            nn.Conv2d(3, base, 5, 2, 2), # H/2, W/2
            nn.ReLU(True),
            nn.Conv2d(base, base * 2, 5, 2, 2), # H/4, W/4
            nn.ReLU(True),
        )
        # Temporal Component: ConvGRU processes the sequence of feature maps
        self.temporal = ConvGRU(base * 2, base * 2)
        # Quantization layer
        self.quant = LatentQuantizer()
        # Spatial Decoder: Restore H/W by 4x
        self.dec = nn.Sequential(
            nn.ConvTranspose2d(base * 2, base, 4, 2, 1), # H/2, W/2
            nn.ReLU(True),
            nn.ConvTranspose2d(base, 3, 4, 2, 1), # H, W
            nn.Sigmoid() # Output pixels in [0, 1]
        )
        # Visual Aid: EQODEC Architecture
        # 

    def forward(self, x):
        B,T,_,H,W = x.shape
        
        # Encode all frames: (B*T, 3, H, W) -> (B*T, C, H/4, W/4)
        f = self.enc(x.view(B*T,3,H,W))
        C,H2,W2 = f.shape[1:]
        
        # ConvGRU: (B, T, C, H/4, W/4) -> (B, T, C, H/4, W/4)
        f = f.view(B,T,C,H2,W2)
        f = self.temporal(f)
        
        # Quantization
        latent = self.quant(f)
        
        # Decode all frames: (B*T, C, H/4, W/4) -> (B*T, 3, H, W)
        out = self.dec(latent.view(B*T,C,H2,W2))
        
        # Final output frame sequence
        out = torch.clamp(out, 0.0, 1.0)
        return out.view(B,T,3,H,W), latent


## --- Loss & Metrics ---
class ReconBPPLoss(nn.Module):
    """MSE + optional bitrate proxy (BPP) for Rate-Distortion trade-off."""
    def __init__(self, lambda_recon=10.0, lambda_bpp=0.0):
        super().__init__()
        self.mse = nn.MSELoss()
        self.lr = lambda_recon
        self.lb = lambda_bpp # Rate (BPP) loss weight

    def forward(self, xhat, x, latent=None):
        mse = self.mse(xhat, x)
        
        bpp = torch.tensor(0.0, device=xhat.device)
        if latent is not None and self.lb > 0:
            # BPP approximation: log2(1 + |latent|) 
            bpp_proxy = torch.mean(torch.log1p(torch.abs(latent))) / math.log(2)
            # Normalize by pixel area (W*H) to get bits per pixel
            bpp = bpp_proxy / (x.shape[-1] * x.shape[-2]) 

        total = self.lr * mse + self.lb * bpp
        return total, {"mse": mse.detach(), "bpp": bpp.detach()}

def compute_psnr(a, b):
    """Computes PSNR from two tensors representing images in [0, 1]."""
    mse = torch.mean((a - b)**2)
    return float(10.0 * torch.log10(1.0 / (mse + 1e-8)))

## --- FFmpeg Helpers ---
def start_ffmpeg_encode_process(out_path, width, height, crf, preset=FFMPEG_PRESET):
    """Start an ffmpeg process that accepts rawvideo rgb24 on stdin."""
    cmd = [
        FFMPEG, "-y",
        "-f","rawvideo","-vcodec","rawvideo","-pix_fmt","rgb24",
        "-s",f"{width}x{height}", "-r","30",
        "-i","pipe:0",
        "-c:v","libx264",
        "-preset",preset,
        "-crf",str(crf),
        out_path
    ]
    p = subprocess.Popen(cmd, stdin=subprocess.PIPE, stderr=subprocess.DEVNULL) 
    return p, time.time()

def close_ffmpeg_process(proc, t_start, out_path):
    """Close stdin, wait for process, return duration and output size."""
    try:
        proc.stdin.close()
    except Exception:
        pass
    ret = proc.wait()
    dur = time.time() - t_start
    if ret != 0:
        raise RuntimeError(f"ffmpeg process failed with return code {ret}")
    size = os.path.getsize(out_path)
    return dur, size

## --- EES Evaluation - CodeCarbon Integrated ---
def evaluate_vimeo_ees(model, loader, device, crf=FFMPEG_CRF, chunk_size=EVAL_CHUNK_SIZE, eval_w=EVAL_RES_W, eval_h=EVAL_RES_H, run_name="eqodec_ees") -> Dict:
    """
    Performs Energy-Efficiency Score (EES) evaluation on the Vimeo test set.
    Measures compressed size and inference/encoding time/carbon cost.
    """
    
    model.eval()
    # Create a temporary directory for FFmpeg output files
    tmp_root = tempfile.mkdtemp(prefix=f"eqodec_eval_vimeo_{run_name}_")

    total_baseline_bytes = 0
    total_model_bytes = 0
    t_baseline = t_model_enc = t_inf = 0.0
    total_psnr = 0.0
    
    if shutil.which(FFMPEG) is None:
        print("[ERROR] FFmpeg not found in system path! EES will fail.")
        return {}
    
    count = 0
    
    # ------------------ CodeCarbon Setup ------------------
    tracker = CarbonModule.tracker(project_name=run_name)
    t_run_start = time.time()
    # ------------------------------------------------------
    
    try:
        # Loop through the entire test loader
        for batch_idx, (frames_full_res, names) in enumerate(tqdm(loader, desc=f"EES {run_name.upper()} on test set")):
            
            x = frames_full_res.to(device)
            B, T, C, H, W = x.shape
            frames_flat = x.view(B * T, C, H, W)
            
            # --- MODEL INFERENCE (Autoencoder Compression/Decompression) ---
            t0_inf = time.time()
            recon_list = []
            
            # Process frames in smaller chunks to manage memory
            for i in range(0, B * T, chunk_size):
                chunk = frames_flat[i:i+chunk_size].unsqueeze(0)
                
                with autocast(device_type=device, enabled=device.startswith("cuda")):
                    with torch.no_grad():
                        recon, _ = model(chunk) 
                        
                clamped_recon = torch.clamp(recon.squeeze(0), 0.0, 1.0)
                clamped_recon = torch.nan_to_num(clamped_recon, nan=0.0, posinf=1.0, neginf=0.0)
                recon_list.append(clamped_recon)

            if device.startswith("cuda"): torch.cuda.synchronize()
            t_inf += time.time() - t0_inf
            
            recon_full = torch.cat(recon_list, dim=0)

            # --- PSNR Calculation ---
            batch_psnr = compute_psnr(recon_full.cpu(), frames_flat.cpu())
            total_psnr += batch_psnr

            # Convert to numpy for FFmpeg encoding
            original_np = (frames_flat.permute(0,2,3,1).cpu().numpy() * 255.0).astype(np.uint8)
            recon_np = (recon_full.permute(0,2,3,1).cpu().numpy() * 255.0).astype(np.uint8)
            
            # --- FFmpeg Encoding Setup ---
            safe_name = f"batch_{batch_idx}"
            out_b = os.path.join(tmp_root, f"{safe_name}_b.mp4") # Baseline (Original)
            out_m = os.path.join(tmp_root, f"{safe_name}_m.mp4") # Model (Reconstructed)
            
            # Note: Baseline uses FFMPEG_CRF, Model uses FFMPEG_MODEL_CRF
            p_base, t0_base = start_ffmpeg_encode_process(out_b, W, H, crf=crf, preset=FFMPEG_PRESET)
            p_model_enc, t0_model_enc = start_ffmpeg_encode_process(out_m, W, H, crf=FFMPEG_MODEL_CRF, preset=FFMPEG_PRESET)

            # --- FFmpeg Encoding Execution ---
            try:
                for frame in original_np:
                    p_base.stdin.write(frame.tobytes())
                    
                for frame in recon_np:
                    p_model_enc.stdin.write(frame.tobytes())

                b_dur, b_size = close_ffmpeg_process(p_base, t0_base, out_b)
                m_dur, m_size = close_ffmpeg_process(p_model_enc, t0_model_enc, out_m)

                t_baseline += b_dur
                t_model_enc += m_dur
                total_baseline_bytes += b_size
                total_model_bytes += m_size

            except Exception as e:
                print(f"\n[ERROR] Crash during FFmpeg encoding for {safe_name}: {e}")
                traceback.print_exc()
                try: p_base.kill(); p_model_enc.kill()
                except Exception: pass
                continue
            
            # Clean up temporary video files
            try:
                os.remove(out_b)
                os.remove(out_m)
            except Exception:
                pass

            count += 1
            
    finally:
        # ------------------ CodeCarbon Teardown ------------------
        try:
            # Stop the tracker and get the total emissions for this evaluation run
            emissions_result = tracker.stop()
            if isinstance(emissions_result, float):
                total_kgco2 = emissions_result
            else:
                total_kgco2 = emissions_result.emissions
        except Exception as e:
            print(f"CodeCarbon tracking failed: {e}")
            total_kgco2 = 0.0
            
        try:
            shutil.rmtree(tmp_root)
        except Exception:
            pass
        # ---------------------------------------------------------

    # Final EES calculations
    t_run_total = time.time() - t_run_start
    # Model overhead is inference time (t_inf) + model-based encoding time (t_model_enc)
    t_model_overhead = t_inf + t_model_enc 
    
    # Delta of compressed size (Original - Model's Compressed Proxy)
    size_delta_gb = (total_baseline_bytes - total_model_bytes) / (1024**3)
    model_size_gb = total_model_bytes / (1024**3) 

    # Calculate Carbon Cost of Model Overhead by proportionality
    if t_run_total > 0 and total_kgco2 > 0:
        # Proportional allocation of the total measured kgCO2 (during the entire run) 
        # to the actual model overhead time (inference + model encoding)
        kg_model_overhead = total_kgco2 * (t_model_overhead / t_run_total)
    else:
        kg_model_overhead = 0.0
    
    # EES: Energy-Efficiency Score (GB saved / kgCO2 cost of overhead)
    # EES is defined as the benefit (size_delta_gb) divided by the cost (kg_model_overhead)
    if kg_model_overhead > 0.0:
        ees = size_delta_gb / kg_model_overhead
    else:
        # If no carbon cost is measured, EES is undefined (nan)
        ees = float("nan")

    avg_psnr = total_psnr / count if count > 0 else float("nan") 

    # Return structured results
    return {
        "model_size_gb": model_size_gb, 
        "t_model_overhead": t_model_overhead, 
        "model_kgco2_overhead": kg_model_overhead, 
        "ees_kgco2_per_gb": ees,
        "avg_psnr_test": avg_psnr, 
        "batches_evaluated": count
    }

## --- Training & Validation ---
# Use the correct, explicit lambda_recon and lambda_rate from the main() scope
def train_one_epoch(model, loader, opt, lambda_c, lambda_rate, lambda_recon, device, carbon_intensity: float, scaler: Optional[GradScaler] = None):
    model.train()
    tot_loss = 0
    tot_psnr = 0
    n = 0
    # Instantiate ReconBPPLoss with correct lambda values
    rec_loss = ReconBPPLoss(lambda_recon=lambda_recon, lambda_bpp=lambda_rate) 
    
    # Visual Aid: Loss Function Components
    # 
    
    for frames, _ in tqdm(loader, desc="Train"):
        x = frames.to(device)

        opt.zero_grad()
        
        # Use AMP if running on CUDA
        with autocast(device_type="cuda", enabled=device.startswith("cuda")):
            xhat, latent = model(x)
            
            # 1. Reconstruction + Rate (BPP) Loss
            loss_recon_rate, diag = rec_loss(xhat, x, latent) 
            
            # 2. Carbon Loss 
            if lambda_c > 0:
                loss_carbon = lambda_c * CarbonModule.carbon_proxy(latent, carbon_intensity)
            else:
                loss_carbon = torch.tensor(0.0, device=xhat.device)

            # 3. Total Loss: L_total = lambda_recon*MSE + lambda_rate*BPP + lambda_c*Carbon_Proxy
            loss = loss_recon_rate + loss_carbon
            
        if scaler is not None:
            scaler.scale(loss).backward()
            scaler.step(opt)
            scaler.update()
        else:
            loss.backward()
            opt.step()

        tot_loss += float(loss.item())
        # Compute PSNR for the first sequence in the batch only for a quick metric
        tot_psnr += compute_psnr(xhat[0], x[0]) 
        n += 1

    return {"train_loss": tot_loss/n, "train_psnr": tot_psnr/n}

@torch.no_grad()
def validate(model, loader, lambda_c, lambda_rate, lambda_recon, device, carbon_intensity: float, scaler: Optional[GradScaler] = None):
    model.eval()
    tot_loss = 0
    tot_psnr = 0
    n = 0
    rec_loss = ReconBPPLoss(lambda_recon=lambda_recon, lambda_bpp=lambda_rate) 
    
    for frames, _ in tqdm(loader, desc="Val"):
        x = frames.to(device)

        with autocast(device_type="cuda", enabled=device.startswith("cuda")):
            xhat, latent = model(x)
            
            # 1. Reconstruction + Rate (BPP) Loss
            loss_recon_rate, diag = rec_loss(xhat, x, latent) 
            
            # 2. Carbon Loss 
            if lambda_c > 0:
                loss_carbon = lambda_c * CarbonModule.carbon_proxy(latent, carbon_intensity)
            else:
                loss_carbon = torch.tensor(0.0, device=xhat.device)
                
            loss = loss_recon_rate + loss_carbon
            
        tot_loss += float(loss.item())
        tot_psnr += compute_psnr(xhat[0], x[0])
        n += 1

    return {"val_loss": tot_loss/n, "val_psnr": tot_psnr/n}


## --- Local Intensity Fetcher ---
def get_local_carbon_intensity(project_name="carbon_rate_check") -> float:
    """
    Runs a small CodeCarbon tracker session to determine the local, 
    average carbon intensity (kgCO2/kWh) for the current region.
    Returns 0.4 kgCO2/kWh if extraction fails.
    """
    print("-> Determining local carbon intensity using CodeCarbon...")
    temp_tracker = EmissionsTracker(
        project_name=project_name,
        log_level="error",
        measure_power_secs=1,
        save_to_file=False
    )
    
    temp_tracker.start()
    
    time.sleep(2) 
    
    emissions_data = temp_tracker.stop()
    
    if isinstance(emissions_data, float):
        print("-> CodeCarbon returned only total emissions (float). Using default 0.4.")
        return 0.4

    try:
        intensity = emissions_data.emissions_rate 
        if intensity is None or intensity < 0.01:
             print("-> Extracted intensity rate was invalid. Using default 0.4.")
             return 0.4
             
        print(f"-> Local Carbon Intensity found: {intensity:.4f} kgCO2/kWh")
        return intensity
        
    except AttributeError:
        print("-> Could not reliably extract emissions_rate from CodeCarbon data. Using default 0.4.")
        return 0.4


## --- ROBUSTNESS IMPLEMENTATION ---
def run_model_replicate(
    run_id: int, 
    model_name: str, 
    lambda_c: float, 
    lambda_rate: float, 
    lambda_recon: float,
    epochs: int, 
    lr: float, 
    train_loader: DataLoader, 
    val_loader: DataLoader, 
    eval_loader: DataLoader, 
    device: str, 
    local_intensity: float
) -> Tuple[Dict, List[Dict]]: 
    """
    Runs a single replicate of training, saves the best model, and evaluates EES
    after every epoch to track learning curves.
    """
    print(f"\n--- Starting {model_name} REPLICATE {run_id}/{N_RUNS} (l_rn={lambda_recon}, l_c={lambda_c}, l_rt={lambda_rate}) ---")
    
    # Save path for the best model of this replicate
    model_path = os.path.join(model_dir, f"{model_name.lower()}_l{lambda_c}_r{run_id}_best.pth")
    
    # Use the defined model architecture
    model = EQODECAutoencoder(base=MODEL_BASE_CHANNELS).to(device)
    opt = optim.Adam(model.parameters(), lr=lr)
    # Initialize GradScaler only if CUDA is used
    scaler = GradScaler() if device.startswith("cuda") else None
    best_loss = float("inf")
    
    # List to collect per-epoch results
    epoch_results_list: List[Dict] = [] 

    # --- Training & Per-Epoch Evaluation ---
    for epoch in range(1, epochs + 1):
        print(f"\n--- Epoch {epoch}/{epochs} ---")
        
        # 1. Training Step
        tm = train_one_epoch(model, train_loader, opt, lambda_c, lambda_rate, lambda_recon, device, local_intensity, scaler=scaler)
        
        # 2. Validation Step
        vm = validate(model, val_loader, lambda_c, lambda_rate, lambda_recon, device, local_intensity, scaler=scaler)
        
        # 3. EES Evaluation (on the current model state)
        ees_results_epoch = evaluate_vimeo_ees(
            model, 
            eval_loader, 
            device, 
            # Note: Baseline always uses FFMPEG_CRF for comparison, Model uses FFMPEG_MODEL_CRF internally
            crf=FFMPEG_CRF, 
            run_name=f"{model_name.lower()}_ees_r{run_id}_ep{epoch}"
        )

        # 4. Consolidate results for the current epoch
        current_epoch_metrics = {
            "epoch": epoch,
            **tm, # train_loss, train_psnr
            **vm, # val_loss, val_psnr
            "ees_kgco2_per_gb": ees_results_epoch.get("ees_kgco2_per_gb", float("nan")),
            "test_psnr": ees_results_epoch.get("avg_psnr_test", float("nan")),
            "test_kgco2_overhead": ees_results_epoch.get("model_kgco2_overhead", float("nan")),
        }
        epoch_results_list.append(current_epoch_metrics)
        
        print(f"Epoch {epoch}: Train Loss={tm['train_loss']:.4f}, Val Loss={vm['val_loss']:.4f}, Test PSNR={current_epoch_metrics['test_psnr']:.4f}, EES={current_epoch_metrics['ees_kgco2_per_gb']:.6f}")
        
        # 5. Save Best Model Checkpoint
        if vm["val_loss"] < best_loss:
            best_loss = vm["val_loss"]
            torch.save(model.state_dict(), model_path)
    
    # --- Final Evaluation (Load Best Model for Final EES) ---
    print(f"\n--- Loading BEST model from epoch with Val Loss {best_loss:.4f} for final EES ---")
    
    # We must load the best model saved during training for the final, official EES score
    model.load_state_dict(torch.load(model_path, map_location=device))
    model.eval()

    # Run EES one last time for the final, best-model result (uses a unique name)
    final_ees_results = evaluate_vimeo_ees(
        model, 
        eval_loader, 
        device, 
        crf=FFMPEG_CRF, 
        run_name=f"{model_name.lower()}_FINAL_ees_r{run_id}_b{MODEL_BASE_CHANNELS}"
    )

    ees_score = final_ees_results.get("ees_kgco2_per_gb", float("nan"))
    print(f"--- {model_name} REPLICATE {run_id} FINAL EES: {ees_score:.6f} ---")
    
    # 6. Save Per-Epoch Results to JSON
    json_path = os.path.join(results_dir, f"{model_name.lower()}_epoch_results_r{run_id}.json")
    with open(json_path, 'w') as f:
        json.dump(epoch_results_list, f, indent=4, default=default_serializer)
    print(f"Per-epoch results saved to: {json_path}")
    
    return final_ees_results, epoch_results_list

def run_robust_evaluation(
    model_name: str, 
    lambda_c: float, 
    lambda_rate: float, 
    lambda_recon: float,
    epochs: int, 
    lr: float, 
    train_loader: DataLoader, 
    val_loader: DataLoader, 
    eval_loader: DataLoader, 
    device: str, 
    local_intensity: float
) -> Dict[str, Dict]:
    """
    Runs multiple replicates and calculates mean and std dev for key metrics.
    Collects the final results from the best model of each replicate.
    Also saves aggregated statistics for easy checkpointing.
    """
    all_results_dicts = []
    
    for i in range(1, N_RUNS + 1):
        final_dict, _ = run_model_replicate(
            run_id=i, 
            model_name=model_name, 
            lambda_c=lambda_c, 
            lambda_rate=lambda_rate,
            lambda_recon=lambda_recon,
            epochs=epochs, 
            lr=lr, 
            train_loader=train_loader, 
            val_loader=val_loader, 
            eval_loader=eval_loader, 
            device=device, 
            local_intensity=local_intensity
        )
        all_results_dicts.append(final_dict)
        
    metrics_to_track = [
        "ees_kgco2_per_gb",
        "avg_psnr_test",
        "t_model_overhead", 
        "model_size_gb", 
        "model_kgco2_overhead"
    ]
    
    aggregated_stats = {}
    for metric in metrics_to_track:
        # Get the final metric from the best model of each run
        values = np.array([res.get(metric, float('nan')) for res in all_results_dicts])
        
        valid_values = values[~np.isnan(values)]

        mean_val = np.mean(valid_values) if valid_values.size > 0 else float("nan")
        std_val = np.std(valid_values) if valid_values.size > 1 else float("nan")
        
        aggregated_stats[metric] = {
            "mean": mean_val,
            "std": std_val,
            "all_scores": values.tolist()
        }
    
    aggregated_stats["avg_psnr"] = aggregated_stats.pop("avg_psnr_test") 

    # --- NEW: Save Aggregated Stats to JSON for Checkpointing ---
    agg_json_path = os.path.join(results_dir, f"{model_name.lower()}_aggregated_stats_b{MODEL_BASE_CHANNELS}.json")
    save_data = {
        "config": {
            "lambda_c": lambda_c, 
            "lambda_rate": lambda_rate, 
            "lambda_recon": lambda_recon,
            "n_runs": N_RUNS,
            "base_channels": MODEL_BASE_CHANNELS,
        },
        "stats": aggregated_stats
    }
    with open(agg_json_path, 'w') as f:
        json.dump(save_data, f, indent=4, default=default_serializer)
    print(f"Aggregated results saved to: {agg_json_path}")
    # --- END NEW ---
    
    return aggregated_stats


## --- MAIN EXECUTION ---
def main():
    
    # --- Paths ---
    data_root = "../data/raw/vimeo/sequences"
    train_idx = "../data/processed/train_split.json"
    val_idx = "../data/processed/val_split.json"
    test_idx = "../data/processed/test_split.json"
    
    # --- Hyperparams ---
    epochs = 10
    lr = 1e-4
    lambda_recon = LAMBDA_RECON
    lambda_c_eqodec = LAMBDA_C_EQODEC 
    lambda_c_baseline = LAMBDA_C_BASELINE # 0.0
    lambda_rate = LAMBDA_RATE 
    lambda_rate_baseline = 0.0 # Baseline should have 0.0 rate loss
    
    device = "cuda" if torch.cuda.is_available() else "cpu"

    # GET THE LOCAL CARBON INTENSITY ONCE
    local_intensity = get_local_carbon_intensity() 

    # --- Loaders ---
    transform = transforms.ToTensor()
    train_loader = DataLoader(VimeoDataset(data_root, train_idx, transform), batch_size=4, shuffle=True)
    val_loader   = DataLoader(VimeoDataset(data_root, val_idx,   transform), batch_size=4, shuffle=False)
    test_loader  = DataLoader(VimeoDataset(data_root, test_idx,  transform), batch_size=4, shuffle=False)

    print(f"\n============================================================")
    print(f"STARTING ROBUST EVALUATION: {N_RUNS} REPLICATES PER MODEL")
    print(f"EQODEC Loss: {lambda_recon}*MSE + {lambda_rate}*BPP + {lambda_c_eqodec}*Carbon_Proxy")
    print(f"Baseline Loss: {lambda_recon}*MSE")
    print(f"Evaluation will be performed on the FULL TEST set, per epoch.")
    print(f"Architecture: ConvGRU, Base Channels={MODEL_BASE_CHANNELS}")
    print(f"Device: {device.upper()}")
    print(f"============================================================")

    # 1. BASELINE ROBUST EVALUATION
    baseline_stats = None
    baseline_agg_path = os.path.join(results_dir, f"baseline_aggregated_stats_b{MODEL_BASE_CHANNELS}.json")

    if os.path.exists(baseline_agg_path):
        print(f"\n--- Loading EXISTING Baseline Stats from: {baseline_agg_path} ---")
        try:
            with open(baseline_agg_path, 'r') as f:
                loaded_data = json.load(f)
                # Simple check to ensure the fundamental model/train configuration matches
                if (loaded_data["config"]["lambda_recon"] == lambda_recon and 
                    loaded_data["config"]["lambda_rate"] == lambda_rate_baseline and
                    loaded_data["config"]["base_channels"] == MODEL_BASE_CHANNELS and
                    loaded_data["config"]["n_runs"] == N_RUNS):
                    
                    baseline_stats = loaded_data["stats"]
                    print("--- Baseline Stats Loaded Successfully. Skipping Retraining. ---")
                else:
                    print("Warning: Loaded Baseline config mismatch. Retraining Baseline.")
        except Exception as e:
            print(f"Error loading or validating Baseline stats ({e}). Retraining Baseline.")
            
    if baseline_stats is None:
        print("\n--- Baseline Stats Not Found or Invalid. STARTING Baseline Training ---")
        baseline_stats = run_robust_evaluation(
            model_name="Baseline",
            lambda_c=lambda_c_baseline, 
            lambda_rate=lambda_rate_baseline,
            lambda_recon=lambda_recon,
            epochs=epochs,
            lr=lr,
            train_loader=train_loader,
            val_loader=val_loader,
            eval_loader=test_loader, 
            device=device,
            local_intensity=local_intensity
        )
    # --- END BASELINE CONDITIONAL EXECUTION ---


    # 2. EQODEC ROBUST EVALUATION
    eqodec_stats = run_robust_evaluation(
        model_name="EQODEC",
        lambda_c=lambda_c_eqodec,
        lambda_rate=lambda_rate,
        lambda_recon=lambda_recon,
        epochs=epochs,
        lr=lr,
        train_loader=train_loader,
        val_loader=val_loader,
        eval_loader=test_loader,
        device=device,
        local_intensity=local_intensity
    )

    # FINAL COMPARISON AND OUTPUT
    results_data = {
        'Metric': [
            'EES',
            'PSNR (Test Set)',
            'Overhead Time',
            'Compressed Size',
            'Total kgCO2 Overhead'
        ],
        'Unit': [
            'GB/kgCO2',
            'dB',
            's',
            'GB',
            'kgCO2'
        ],
        'EQODEC (Mean ± Std)': [
            f"{eqodec_stats['ees_kgco2_per_gb']['mean']:.6f} ± {eqodec_stats['ees_kgco2_per_gb']['std']:.6f}",
            f"{eqodec_stats['avg_psnr']['mean']:.4f} ± {eqodec_stats['avg_psnr']['std']:.4f}",
            f"{eqodec_stats['t_model_overhead']['mean']:.6f} ± {eqodec_stats['t_model_overhead']['std']:.6f}",
            f"{eqodec_stats['model_size_gb']['mean']:.6f} ± {eqodec_stats['model_size_gb']['std']:.6f}",
            f"{eqodec_stats['model_kgco2_overhead']['mean']:.6f} ± {eqodec_stats['model_kgco2_overhead']['std']:.6f}"
        ],
        'Baseline (Mean ± Std)': [
            f"{baseline_stats['ees_kgco2_per_gb']['mean']:.6f} ± {baseline_stats['ees_kgco2_per_gb']['std']:.6f}",
            f"{baseline_stats['avg_psnr']['mean']:.4f} ± {baseline_stats['avg_psnr']['std']:.4f}",
            f"{baseline_stats['t_model_overhead']['mean']:.6f} ± {baseline_stats['t_model_overhead']['std']:.6f}",
            f"{baseline_stats['model_size_gb']['mean']:.6f} ± {baseline_stats['model_size_gb']['std']:.6f}",
            f"{baseline_stats['model_kgco2_overhead']['mean']:.6f} ± {baseline_stats['model_kgco2_overhead']['std']:.6f}"
        ]
    }
    
    df_results = pd.DataFrame(results_data)
    
    csv_path = os.path.join(results_dir, "robust_ees_comparison_test_set_rd.csv")
    df_results.to_csv(csv_path, index=False)
    
    print("\n============================================================")
    print(f"FINAL ROBUST COMPARISON (N={N_RUNS} REPLICATES) on FULL TEST SET")
    print("============================================================")
    print(df_results.to_string(index=False))
    print(f"\nFinal mean/std results saved to CSV: {csv_path}")

    print("\n--- All EES Scores (for Variance Analysis) ---")
    print(f"EQODEC All EES: {eqodec_stats['ees_kgco2_per_gb']['all_scores']}")
    print(f"Baseline All EES: {baseline_stats['ees_kgco2_per_gb']['all_scores']}")
    print("============================================================")

In [3]:
main()

[codecarbon WARNING @ 16:43:49] Multiple instances of codecarbon are allowed to run at the same time.


-> Determining local carbon intensity using CodeCarbon...
-> CodeCarbon returned only total emissions (float). Using default 0.4.

STARTING ROBUST EVALUATION: 5 REPLICATES PER MODEL
EQODEC Loss: 5.0*MSE + 0.5*BPP + 0.005*Carbon_Proxy
Baseline Loss: 5.0*MSE
Evaluation will be performed on the FULL TEST set, per epoch.
Architecture: ConvGRU, Base Channels=16
Device: CUDA

--- Baseline Stats Not Found or Invalid. STARTING Baseline Training ---

--- Starting Baseline REPLICATE 1/5 (l_rn=5.0, l_c=0.0, l_rt=0.0) ---

--- Epoch 1/10 ---


EES BASELINE_EES_R1_EP1 on test set: 100%|█████████████████████████████████████████████| 24/24 [00:11<00:00,  2.05it/s]


Epoch 1: Train Loss=0.4600, Val Loss=0.4428, Test PSNR=10.6921, EES=3.082585

--- Epoch 2/10 ---


EES BASELINE_EES_R1_EP2 on test set: 100%|█████████████████████████████████████████████| 24/24 [00:12<00:00,  1.92it/s]


...

Epoch 9: Train Loss=0.0788, Val Loss=0.0843, Test PSNR=17.4518, EES=2.153401

--- Epoch 10/10 ---


EES EQODEC_EES_R5_EP10 on test set: 100%|██████████████████████████████████████████████| 24/24 [00:12<00:00,  1.95it/s]


Epoch 10: Train Loss=0.0724, Val Loss=0.0789, Test PSNR=17.7727, EES=2.146918

--- Loading BEST model from epoch with Val Loss 0.0789 for final EES ---


EES EQODEC_FINAL_EES_R5_B16 on test set: 100%|█████████████████████████████████████████| 24/24 [00:12<00:00,  1.95it/s]

--- EQODEC REPLICATE 5 FINAL EES: 2.145486 ---
Per-epoch results saved to: ../results\eqodec_epoch_results_r5.json
Aggregated results saved to: ../results\eqodec_aggregated_stats_b16.json

FINAL ROBUST COMPARISON (N=5 REPLICATES) on FULL TEST SET
              Metric     Unit EQODEC (Mean ± Std) Baseline (Mean ± Std)
                 EES GB/kgCO2 2.182022 ± 0.122040   2.083198 ± 0.030465
     PSNR (Test Set)       dB    16.1505 ± 1.6436      17.1532 ± 0.4132
       Overhead Time        s 7.002199 ± 0.168555   6.986921 ± 0.035673
     Compressed Size       GB 0.000366 ± 0.000045   0.000418 ± 0.000011
Total kgCO2 Overhead    kgCO2 0.000514 ± 0.000012   0.000513 ± 0.000003

Final mean/std results saved to CSV: ../results\robust_ees_comparison_test_set_rd.csv

--- All EES Scores (for Variance Analysis) ---
EQODEC All EES: [2.102729384804174, 2.202605248163199, 2.4055520204637153, 2.0537344617978373, 2.1454864103467246]
Baseline All EES: [2.0534294920522713, 2.088505258221711, 2.04447132727

**MODEL OUTPUT SAMPLE**

In [4]:
def save_comparison_frames(
    model_base_channels: int, 
    lambda_c_eqodec: float, 
    lambda_c_baseline: float, 
    model_dir: str, 
    data_root: str, 
    test_idx: str,
    output_dir: str = "./visual_comparison_frames"
):
    """
    Loads trained models, runs inference on a single sequence, and saves the 
    Original, EQODEC, and Baseline reconstructed frames to a directory.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    
    # Paths (Assuming run_id=1 was successfully saved)
    eqodec_path = os.path.join(model_dir, f"eqodec_l{lambda_c_eqodec}_r1_best.pth")
    base_path = os.path.join(model_dir, f"baseline_l{lambda_c_baseline}_r1_best.pth")
    
    # --- Helper to convert tensor (C, H, W) to PIL Image ---
    def tensor_to_pil(tensor: torch.Tensor) -> Image:
        # Detach, move to CPU, convert C,H,W to H,W,C, convert to 0-255 uint8
        img_np = (tensor.detach().cpu().permute(1, 2, 0).numpy() * 255).astype(np.uint8)
        return Image.fromarray(img_np)

    try:
        # --- Load Models ---
        # ... (Model loading code remains the same) ...
        print(f"Loading models for image output...")
        model_eqodec = EQODECAutoencoder(base=model_base_channels).to(device).eval()
        model_baseline = EQODECAutoencoder(base=model_base_channels).to(device).eval()

        model_eqodec.load_state_dict(torch.load(eqodec_path, map_location=device))
        model_baseline.load_state_dict(torch.load(base_path, map_location=device))
        
        # --- Data Loader Setup ---
        transform = transforms.ToTensor()
        test_loader = DataLoader(VimeoDataset(data_root, test_idx, transform), batch_size=1, shuffle=False)
        
        # --- Get a single test sequence ---
        frames, seq_name = next(iter(test_loader))
        x = frames.to(device) 
        B, T, C, H, W = x.shape
        
        # FIX 1: Clean the sequence name to replace illegal separators ('/') 
        # with a safe character (e.g., '_') for file system compatibility.
        sequence_id = seq_name[0].replace(os.path.sep, '_').replace('/', '_')
        print(f"Processing sequence: {sequence_id} ({T} frames)")
        
        # FIX 2: Create the base output directory before saving
        os.makedirs(output_dir, exist_ok=True)


        # --- Run Inference ---
        with torch.no_grad():
            with autocast(device_type="cuda", enabled=device.startswith("cuda")):
                recon_eqodec, _ = model_eqodec(x)
                recon_baseline, _ = model_baseline(x)
        
        # --- Memory Cleanup ---
        if device == "cuda":
            del model_eqodec, model_baseline
            torch.cuda.empty_cache() 
            
        # --- Save Frames ---
        for i in range(T):
            frame_suffix = f"f{i:02d}.png"
            
            # Save Original
            pil_img_orig = tensor_to_pil(x[0, i])
            orig_path = os.path.join(output_dir, f"{sequence_id}_Original_{frame_suffix}")
            pil_img_orig.save(orig_path)

            # Save EQODEC
            pil_img_eqodec = tensor_to_pil(recon_eqodec[0, i])
            eqodec_path_out = os.path.join(output_dir, f"{sequence_id}_EQODEC_{frame_suffix}")
            pil_img_eqodec.save(eqodec_path_out)

            # Save Baseline
            pil_img_baseline = tensor_to_pil(recon_baseline[0, i])
            baseline_path_out = os.path.join(output_dir, f"{sequence_id}_Baseline_{frame_suffix}")
            pil_img_baseline.save(baseline_path_out)

        print(f"\nSuccessfully saved {T} frames for Original, EQODEC, and Baseline to {output_dir}/")
        
    except Exception as e:
        print(f"\n[CRITICAL ERROR] Failed to save comparison frames: {e}")
        print("Ensure all required classes/constants are defined and model files exist.")

In [5]:
# SAVE IMAGES TO DISK
save_comparison_frames(
    model_base_channels=MODEL_BASE_CHANNELS,
    lambda_c_eqodec=LAMBDA_C_EQODEC,
    lambda_c_baseline=LAMBDA_C_BASELINE,
    model_dir="../models",
    data_root="../data/raw/vimeo/sequences",
    test_idx="../data/processed/test_split.json",
    output_dir="../results/visual_comparison_frames"
)

Loading models for image output...
Processing sequence: 00007_0053 (7 frames)

Successfully saved 7 frames for Original, EQODEC, and Baseline to ../results/visual_comparison_frames/
